In [ ]:
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
import pandas as pd
df=pd.read_csv('/kaggle/input/scor-dataset/updated_scor_raw_df (22).csv')
# Step 1: Select one SiteID with sufficient data
site_data = df[df['SiteID'] == df['SiteID'].value_counts().idxmax()]

# Step 2: Aggregate scor_lta by year (average per year per site)
ts = site_data.groupby('Year')['scor_lta'].mean().sort_index()

# Step 3: Fit ARIMA model (order to be selected based on simplicity for now)
model = ARIMA(ts, order=(1,1,1))
fitted_model = model.fit()

# Step 4: Forecast next 5 years
forecast = fitted_model.forecast(steps=5)

# Plotting
plt.figure(figsize=(10,5))
plt.plot(ts, label='Observed')
plt.plot(range(ts.index.max()+1, ts.index.max()+6), forecast, label='Forecast', linestyle='--')
plt.xlabel("Year")
plt.ylabel("Average scor_lta")
plt.title("ARIMA Forecast for Live Tissue Area (scor_lta)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt

# Load your dataset

# Filter for a single SiteID
site_df = df[df["SiteID"] == 18]

# Group by year and take mean of scor_lta
prophet_df = site_df.groupby("Year")["scor_lta"].mean().reset_index()
prophet_df = prophet_df.rename(columns={"Year": "ds", "scor_lta": "y"})
prophet_df["ds"] = pd.to_datetime(prophet_df["ds"], format="%Y")

# Initialize Prophet model
model = Prophet(yearly_seasonality=False)
model.fit(prophet_df)

# Forecast next 5 years
future = model.make_future_dataframe(periods=5, freq='Y')
forecast = model.predict(future)

# Plot forecast
fig1 = model.plot(forecast)
fig1.suptitle("Prophet Forecast for scor_lta (SiteID 18)", fontsize=14)

# Plot trend and components
fig2 = model.plot_components(forecast)
fig2.suptitle("Prophet Components (Trend, Seasonality)", fontsize=14)

plt.show()


In [ ]:
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt

# Use the overall trend data again
ts_overall = df.groupby("Year")["scor_lta"].mean().sort_index()
ts_overall.index = pd.to_datetime(ts_overall.index, format="%Y")

# Fit ARIMA model (basic example with order=(1,1,1))
model = ARIMA(ts_overall, order=(1, 1, 1))
fitted_model = model.fit()

# Forecast next 5 years
forecast_arima = fitted_model.forecast(steps=5)

# Prepare future years for x-axis
future_years = pd.date_range(start=ts_overall.index[-1] + pd.DateOffset(years=1), periods=5, freq='Y')

# Plot the results
plt.figure(figsize=(10, 5))
plt.plot(ts_overall, label='Observed')
plt.plot(future_years, forecast_arima, label='Forecast (ARIMA)', linestyle='--')
plt.title("ARIMA Forecast: General Trend of Coral scor_lta Across All Sites")
plt.xlabel("Year")
plt.ylabel("Average scor_lta")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt

# Load your data
df = pd.read_csv("/kaggle/input/scor-dataset/updated_scor_raw_df (22).csv")

# Compute average scor_lta per year across all sites
overall_trend_df = df.groupby("Year")["scor_lta"].mean().reset_index()
overall_trend_df = overall_trend_df.rename(columns={"Year": "ds", "scor_lta": "y"})
overall_trend_df["ds"] = pd.to_datetime(overall_trend_df["ds"], format="%Y")

# Initialize Prophet
model = Prophet(yearly_seasonality=False)
model.fit(overall_trend_df)

# Forecast next 5 years
future = model.make_future_dataframe(periods=5, freq='Y')
forecast = model.predict(future)

# Plot forecast
fig1 = model.plot(forecast)
fig1.suptitle("Prophet Forecast: General Trend of Coral scor_lta Across All Sites", fontsize=14)

# Plot trend components
fig2 = model.plot_components(forecast)
fig2.suptitle("Prophet Components (General Trend)", fontsize=14)

plt.show()


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Masking

# Load data
df = pd.read_csv("/kaggle/input/scor-dataset/updated_scor_raw_df (22).csv")

# Aggregate by SiteID and Year
agg_df = df.groupby(['SiteID', 'Year']).agg({
    'scor_lta': 'mean',
    'TempC': 'mean'
}).reset_index()

# Handle missing values
agg_df = agg_df.dropna(subset=['scor_lta'])
agg_df['TempC'] = agg_df.groupby('SiteID')['TempC'].transform(lambda x: x.fillna(x.mean()))
agg_df['TempC'] = agg_df['TempC'].fillna(agg_df['TempC'].mean())

# Convert to sequences per site
site_sequences = []
for _, group in agg_df.groupby('SiteID'):
    group = group.sort_values('Year')
    site_sequences.append(group[['scor_lta', 'TempC']].values)

# Prepare padded input/output sequences
X, y = [], []
for seq in site_sequences:
    if len(seq) < 3:
        continue
    X.append(seq[:-1])
    y.append(seq[-1][0])  # predict scor_lta for last year

max_seq_len = max(len(s) for s in X)
X_padded = pad_sequences(X, maxlen=max_seq_len, padding='pre', dtype='float32')

# Normalize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_padded.reshape(-1, 2)).reshape(X_padded.shape)

# Split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, np.array(y), test_size=0.2, random_state=42)

# Define LSTM model
def build_lstm():
    model = Sequential([
        Masking(mask_value=0.0, input_shape=(X_train.shape[1], 2)),
        LSTM(64),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Define GRU model
def build_gru():
    model = Sequential([
        Masking(mask_value=0.0, input_shape=(X_train.shape[1], 2)),
        GRU(64),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Train LSTM
lstm_model = build_lstm()
lstm_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, verbose=2)

# Train GRU
gru_model = build_gru()
gru_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, verbose=2)


In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout, Masking
from tqdm.keras import TqdmCallback
import math

# Load data
df = pd.read_csv("/kaggle/input/scor-dataset/updated_scor_raw_df (22).csv")

# Aggregate and select features
agg_df = df.groupby(['SiteID', 'Year']).agg({
    'scor_lta': 'mean',
    'TempC': 'mean',
    'density': 'mean',
    'LTA_cm2': 'mean'
}).reset_index()

# Handle missing
agg_df = agg_df.dropna(subset=['scor_lta'])
for col in ['TempC', 'density', 'LTA_cm2']:
    agg_df[col] = agg_df.groupby('SiteID')[col].transform(lambda x: x.fillna(x.mean()))
    agg_df[col] = agg_df[col].fillna(agg_df[col].mean())

# Prepare rolling window sequences
X, y = [], []
window = 5  # sequence length

for _, group in agg_df.groupby("SiteID"):
    group = group.sort_values("Year")
    data = group[['scor_lta', 'TempC', 'density', 'LTA_cm2']].values

    for i in range(len(data) - window):
        X.append(data[i:i+window])
        y.append(np.log1p(data[i+window][0]))  # log-transform scor_lta target

# Pad and normalize
X = np.array(X)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X.reshape(-1, 4)).reshape(X.shape)

# Split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, np.array(y), test_size=0.2, random_state=42)

# Define LSTM model
def build_lstm():
    model = Sequential([
        Masking(mask_value=0.0, input_shape=(X_train.shape[1], X_train.shape[2])),
        LSTM(64, return_sequences=True),
        Dropout(0.3),
        LSTM(64),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Define GRU model
def build_gru():
    model = Sequential([
        Masking(mask_value=0.0, input_shape=(X_train.shape[1], X_train.shape[2])),
        GRU(64, return_sequences=True),
        Dropout(0.3),
        GRU(64),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Train LSTM
print("Training LSTM model...")
lstm_model = build_lstm()
lstm_model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=50,
    verbose=0,
    callbacks=[TqdmCallback(verbose=1)]
)

# Evaluate LSTM
lstm_preds = lstm_model.predict(X_test)
lstm_preds = np.expm1(lstm_preds).flatten()
y_true = np.expm1(y_test)

lstm_mae = mean_absolute_error(y_true, lstm_preds)
lstm_rmse = math.sqrt(mean_squared_error(y_true, lstm_preds))
print(f"\n📈 LSTM MAE: {lstm_mae:.2f}, RMSE: {lstm_rmse:.2f}")

# Train GRU
print("\nTraining GRU model...")
gru_model = build_gru()
gru_model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=50,
    verbose=0,
    callbacks=[TqdmCallback(verbose=1)]
)

# Evaluate GRU
gru_preds = gru_model.predict(X_test)
gru_preds = np.expm1(gru_preds).flatten()

gru_mae = mean_absolute_error(y_true, gru_preds)
gru_rmse = math.sqrt(mean_squared_error(y_true, gru_preds))
print(f"\n📈 GRU MAE: {gru_mae:.2f}, RMSE: {gru_rmse:.2f}")
from sklearn.metrics import r2_score

def smape(y_true, y_pred):
    return 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

def mape(y_true, y_pred):
    return 100 * np.mean(np.abs((y_true - y_pred) / y_true))

# Already computed y_true, lstm_preds, gru_preds

print("\n🔍 LSTM Metrics:")
print(f"MAE  : {mean_absolute_error(y_true, lstm_preds):.2f}")
print(f"RMSE : {math.sqrt(mean_squared_error(y_true, lstm_preds)):.2f}")
print(f"MAPE : {mape(y_true, lstm_preds):.2f}%")
print(f"SMAPE: {smape(y_true, lstm_preds):.2f}%")
print(f"R²   : {r2_score(y_true, lstm_preds):.4f}")

print("\n🔍 GRU Metrics:")
print(f"MAE  : {mean_absolute_error(y_true, gru_preds):.2f}")
print(f"RMSE : {math.sqrt(mean_squared_error(y_true, gru_preds)):.2f}")
print(f"MAPE : {mape(y_true, gru_preds):.2f}%")
print(f"SMAPE: {smape(y_true, gru_preds):.2f}%")
print(f"R²   : {r2_score(y_true, gru_preds):.4f}")


2025-04-25 13:58:38.326726: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745589518.610026      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745589518.690162      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Training LSTM model...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
I0000 00:00:1745589532.061336      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14955 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

...

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step  

📈 LSTM MAE: 2196.91, RMSE: 3473.00

Training GRU model...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step  

📈 GRU MAE: 1963.03, RMSE: 2885.25

🔍 LSTM Metrics:
MAE  : 2196.91
RMSE : 3473.00
MAPE : 48.78%
SMAPE: 34.72%
R²   : 0.7893

🔍 GRU Metrics:
MAE  : 1963.03
RMSE : 2885.25
MAPE : 44.55%
SMAPE: 31.62%
R²   : 0.8545


In [5]:
import pandas as pd
import numpy as np
import torch
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE
from pytorch_lightning import Trainer, seed_everything, LightningModule
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import math
import types

# ⚙️ Load dataset
df = pd.read_csv("/kaggle/input/scor-dataset/updated_scor_raw_df (22).csv")

# 🧼 Preprocess
agg_df = df.groupby(['SiteID', 'Year']).agg({
    'scor_lta': 'mean',
    'TempC': 'mean',
    'density': 'mean',
    'LTA_cm2': 'mean'
}).reset_index()

agg_df = agg_df.dropna(subset=['scor_lta'])

for col in ['TempC', 'density', 'LTA_cm2']:
    agg_df[col] = agg_df.groupby("SiteID")[col].transform(lambda x: x.fillna(x.mean()))
    agg_df[col] = agg_df[col].fillna(agg_df[col].mean())

agg_df['scor_lta'] = np.log1p(agg_df['scor_lta'])  # log transform
agg_df["time_idx"] = agg_df["Year"] - agg_df["Year"].min()
agg_df["SiteID"] = agg_df["SiteID"].astype(str)

# ⏳ Temporal params
max_encoder_length = 3
max_prediction_length = 1

site_counts = agg_df.groupby("SiteID")["Year"].nunique()
valid_sites = site_counts[site_counts >= (max_encoder_length + max_prediction_length)].index
agg_df = agg_df[agg_df["SiteID"].isin(valid_sites)]

# 📦 Dataset
training = TimeSeriesDataSet(
    agg_df,
    time_idx="time_idx",
    target="scor_lta",
    group_ids=["SiteID"],
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    static_categoricals=["SiteID"],
    time_varying_known_reals=["time_idx", "Year"],
    time_varying_unknown_reals=["scor_lta", "TempC", "density", "LTA_cm2"],
    target_normalizer=GroupNormalizer(groups=["SiteID"]),
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
    allow_missing_timesteps=True
)

# ✂️ Train/Val split
val_cutoff = agg_df["time_idx"].max() - max_prediction_length
train_data = agg_df[agg_df["time_idx"] <= val_cutoff]
val_data = agg_df[agg_df["time_idx"] > val_cutoff]

def filter_min_years(df, min_years=4):
    site_counts = df.groupby("SiteID")["Year"].nunique()
    valid_sites = site_counts[site_counts >= min_years].index
    return df[df["SiteID"].isin(valid_sites)]

train_data = filter_min_years(train_data)
val_data = filter_min_years(val_data)

# 🔄 Dataloaders
train_ds = TimeSeriesDataSet.from_dataset(training, train_data)
train_dataloader = train_ds.to_dataloader(train=True, batch_size=64, num_workers=0)

val_dataloader = None
if len(val_data) >= 1:
    val_ds = TimeSeriesDataSet.from_dataset(training, val_data)
    val_dataloader = val_ds.to_dataloader(train=False, batch_size=64, num_workers=0)
else:
    print("⚠️ Validation set is empty after filtering. Proceeding without validation.")

# ✅ Lightning wrapper for PL 2.x
class TFTLightningWrapper(LightningModule):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def training_step(self, batch, batch_idx):
        loss = self.model.training_step(batch, batch_idx)
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        loss = self.model.validation_step(batch, batch_idx)
        self.log("val_loss", loss)
        return loss

    def configure_optimizers(self):
        return self.model.configure_optimizers()

    def forward(self, *args, **kwargs):
        return self.model(*args, **kwargs)

# 🎯 Instantiate raw TFT model
seed_everything(42)

tft_raw = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.03,
    hidden_size=32,
    attention_head_size=1,
    dropout=0.1,
    hidden_continuous_size=16,
    output_size=1,
    loss=SMAPE(),
    logging_metrics=None,
)

# ✅ Patch: fix device mismatch in get_attention_mask
def patched_get_attention_mask(self, encoder_lengths, decoder_lengths):
    encoder_mask = torch.arange(self.max_encoder_length, device=encoder_lengths.device)[None, :] < encoder_lengths[:, None]
    decoder_mask = torch.arange(self.max_prediction_length, device=decoder_lengths.device)[None, :] < decoder_lengths[:, None]
    return torch.cat((encoder_mask, decoder_mask), dim=1)

tft_raw.get_attention_mask = types.MethodType(patched_get_attention_mask, tft_raw)

# ✅ Wrap for Lightning 2.x training
tft = TFTLightningWrapper(tft_raw)

# ⚡ Trainer
trainer = Trainer(
    max_epochs=30,
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    devices=1 if torch.cuda.is_available() else None,
    gradient_clip_val=0.1,
    enable_progress_bar=True
)

# 🔁 Fit model
trainer.fit(tft, train_dataloader, val_dataloader if val_dataloader else None)

# 📊 Evaluate if possible
if val_dataloader:
    predictions, x = tft_raw.predict(val_dataloader, return_x=True)
    y_true_log = x["decoder_target"].detach().cpu().numpy()
    y_pred_log = predictions.detach().cpu().numpy()

    y_true = np.expm1(y_true_log).flatten()
    y_pred = np.expm1(y_pred_log).flatten()

    def smape_score(y_true, y_pred):
        return 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

    def mape_score(y_true, y_pred):
        return 100 * np.mean(np.abs((y_true - y_pred) / y_true))


ModuleNotFoundError: No module named 'pytorch_forecasting'

In [ ]:
! pip install pytorch_forecasting

In [ ]:
pip install pytorch-lightning==2.1.3 pytorch-forecasting==1.3.0 optuna tqdm --quiet


In [ ]:
pip install pytorch-lightning==2.1.3 pytorch-forecasting==1.3.0 optuna tqdm --quiet


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the condition dataset
df = pd.read_csv("/kaggle/input/condition-counts/CREMP_SCOR_Summaries_2023_ConditionCounts.csv")

# Step 1: Melt condition columns for long-format analysis
condition_cols = df.columns[9:]  # All condition columns
melted = df.melt(
    id_vars=['SiteID', 'Site_name', 'sciName'],
    value_vars=condition_cols,
    var_name='Condition',
    value_name='ConditionCount'  # <- Changed from 'Count' to avoid conflict
)

# Step 2: Remove 0 or NaN condition counts
melted['ConditionCount'] = pd.to_numeric(melted['ConditionCount'], errors='coerce')
melted = melted[melted['ConditionCount'] > 0]

# Step 3: Top conditions overall
top_conditions = melted.groupby('Condition')['ConditionCount'].sum().sort_values(ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x=top_conditions.values, y=top_conditions.index, palette='viridis')
plt.title("Most Frequently Observed Coral Conditions")
plt.xlabel("Total Observations")
plt.ylabel("Condition")
plt.tight_layout()
plt.show()

# Step 4: Heatmap of conditions per site
site_condition = melted.groupby(['Site_name', 'Condition'])['ConditionCount'].sum().reset_index()
pivot_site = site_condition.pivot(index='Site_name', columns='Condition', values='ConditionCount').fillna(0)

plt.figure(figsize=(15, 10))
sns.heatmap(pivot_site, cmap="YlOrRd", linewidths=0.5)
plt.title("Condition Distribution Across Coral Sites")
plt.xlabel("Condition")
plt.ylabel("Site")
plt.tight_layout()
plt.show()

# Step 5: Heatmap of conditions per species
species_condition = melted.groupby(['sciName', 'Condition'])['ConditionCount'].sum().reset_index()
pivot_species = species_condition.pivot(index='sciName', columns='Condition', values='ConditionCount').fillna(0)

plt.figure(figsize=(15, 12))
sns.heatmap(pivot_species, cmap="Blues", linewidths=0.5)
plt.title("Condition Impact Across Coral Species")
plt.xlabel("Condition")
plt.ylabel("Coral Species")
plt.tight_layout()
plt.show()


In [ ]:
import folium
from folium.plugins import MarkerCluster
import seaborn as sns

# Load condition count and station coordinates
condition_df = pd.read_csv('/kaggle/input/condition-counts/CREMP_SCOR_Summaries_2023_ConditionCounts.csv')
coords_df = pd.read_csv('/kaggle/input/coral-reef-data/coral_csv/CREMP_Stations_2023.csv')

# Ensure numeric columns are properly typed
for col in condition_df.columns[9:]:
    condition_df[col] = pd.to_numeric(condition_df[col], errors='coerce')

# Compute a severity score (simple sum of all conditions per SiteID)
condition_df['SeverityScore'] = condition_df.iloc[:, 9:].sum(axis=1)
site_severity = condition_df.groupby('SiteID')['SeverityScore'].sum().reset_index()

# Merge with coordinates
merged_df = coords_df.merge(site_severity, on='SiteID', how='left').fillna(0)

# Color palette based on severity (normalize scores)
max_score = merged_df['SeverityScore'].max()
merged_df['Color'] = merged_df['SeverityScore'].apply(lambda x: sns.color_palette("Reds", as_cmap=True)(x / max_score))

# Create folium map
reef_map = folium.Map(location=[merged_df['latDD'].mean(), merged_df['lonDD'].mean()],
                      zoom_start=8, tiles="cartodbpositron")
marker_cluster = MarkerCluster().add_to(reef_map)

# Add each site to map with severity-based circle marker
for _, row in merged_df.iterrows():
    folium.CircleMarker(
        location=[row['latDD'], row['lonDD']],
        radius=6 + (row['SeverityScore'] / max_score) * 10,  # bubble size scaled by severity
        color=None,
        fill=True,
        fill_color=sns.color_palette("Reds", as_cmap=True)(row['SeverityScore'] / max_score),
        fill_opacity=0.8,
        popup=(f"<b>Site:</b> {row['Site_name']}<br>"
               f"<b>Habitat:</b> {row['Habitat']}<br>"
               f"<b>Severity Score:</b> {row['SeverityScore']:.1f}<br>"
               f"<b>Depth:</b> {row['Depth_ft']} ft"),
    ).add_to(marker_cluster)
reef_map


In [ ]:
import folium
from folium.plugins import MarkerCluster
from branca.colormap import LinearColormap

# Setup severity color scale
severity_max = merged_df['SeverityScore'].max()
severity_min = merged_df['SeverityScore'].min()
colormap = LinearColormap(
    colors=['green', 'yellow', 'orange', 'red', 'darkred'],
    index=[severity_min, severity_max * 0.25, severity_max * 0.5, severity_max * 0.75, severity_max],
    vmin=severity_min,
    vmax=severity_max,
    caption='Coral Condition Severity Score'
)

# Create map
reef_map = folium.Map(location=[merged_df['latDD'].mean(), merged_df['lonDD'].mean()],
                      zoom_start=8, tiles='cartodbpositron')

marker_cluster = MarkerCluster().add_to(reef_map)

# Add markers with color/size based on severity
for _, row in merged_df.iterrows():
    severity = row['SeverityScore']
    folium.CircleMarker(
        location=[row['latDD'], row['lonDD']],
        radius=4 + (severity / severity_max) * 10,
        color=colormap(severity),
        fill=True,
        fill_color=colormap(severity),
        fill_opacity=0.85,
        popup=folium.Popup(
            html=(
                f"<b>📍 Site:</b> {row['Site_name']}<br>"
                f"<b>🏝️ Habitat:</b> {row['Habitat']}<br>"
                f"<b>🌡️ Depth:</b> {row['Depth_ft']} ft<br>"
                f"<b>🔥 Severity Score:</b> <b style='color:{colormap(severity)}'>{severity:.1f}</b>"
            ),
            max_width=250
        )
    ).add_to(marker_cluster)

# Add color scale to map
colormap.add_to(reef_map)

reef_map


In [ ]:
import pandas as pd
import folium
from folium.plugins import HeatMap

# --- STEP 1: Load and process data ---



# Convert all condition columns to numeric safely
for col in condition_df.columns[9:]:
    condition_df[col] = pd.to_numeric(condition_df[col], errors='coerce')

# Calculate severity score = sum of all condition counts
condition_df['SeverityScore'] = condition_df.iloc[:, 9:].sum(axis=1)

# Aggregate severity by SiteID
site_severity = condition_df.groupby('SiteID')['SeverityScore'].sum().reset_index()

# Merge severity scores with station coordinates
merged_df = coords_df.merge(site_severity, on='SiteID', how='left').fillna(0)

# --- STEP 2: Prepare data for heatmap ---

# Extract necessary columns and drop rows with missing values
heatmap_df = merged_df[['latDD', 'lonDD', 'SeverityScore']].dropna()

# Ensure correct column names and datatypes
heatmap_df.columns = heatmap_df.columns.map(str)
heatmap_df = heatmap_df.astype(float)

# Create list of [lat, lon, severity] points for the heatmap
heat_data = heatmap_df[['latDD', 'lonDD', 'SeverityScore']].values.tolist()

# --- STEP 3: Generate Folium HeatMap ---

# Create base map centered on the region
heat_map = folium.Map(
    location=[heatmap_df['latDD'].mean(), heatmap_df['lonDD'].mean()],
    zoom_start=7,
    tiles="cartodb dark_matter"
)

# Add heatmap layer
HeatMap(
    data=heat_data,
    radius=18,
    blur=15,
    max_zoom=13,
    min_opacity=0.4,
    gradient={
        0.2: 'green',
        0.4: 'yellow',
        0.6: 'orange',
        0.8: 'red',
        1.0: 'darkred'
    }
).add_to(heat_map)

# Show the map in the notebook
heat_map


In [ ]:
pip install biopython


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 21.5 MB/s eta 0:00:0000:0100:01


In [2]:
!pip install biopython
from Bio import SeqIO
import pandas as pd

records = list(SeqIO.parse("/kaggle/input/fastq-output/fastq_output/SRR20727752_1.fastq", "fastq"))
df = pd.DataFrame({
    "Read_ID": [rec.id for rec in records],
    "Sequence": [str(rec.seq) for rec in records],
    "Quality_Scores": [rec.letter_annotations["phred_quality"] for rec in records],
    "Avg_Quality": [sum(rec.letter_annotations["phred_quality"])/len(rec) for rec in records],
    "Length": [len(rec.seq) for rec in records]
})
df.to_csv("SRR20727787_2_converted.csv", index=False)


In [3]:
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

# Load the FASTQ CSV file
df = pd.read_csv("/kaggle/working/SRR20727787_2_converted.csv")  # <- update path if needed

# Label: Favorable = Avg_Quality > 30
df['Label'] = (df['Avg_Quality'] > 30).astype(int)

# Encode DNA sequence (A=0, T=1, G=2, C=3, N=4)
vocab = {'A': 0, 'T': 1, 'G': 2, 'C': 3, 'N': 4}
def encode_sequence(seq, max_len=300):
    return [vocab.get(base, 4) for base in seq[:max_len]] + [0]*(max_len - len(seq))

df['Encoded_Seq'] = df['Sequence'].apply(lambda x: encode_sequence(x))

# Split into train/test
X_train, X_test, y_train, y_test = train_test_split(df['Encoded_Seq'].tolist(), df['Label'].tolist(), test_size=0.2)

# Torch Dataset
class DNADataset(Dataset):
    def __init__(self, sequences, labels):
        self.X = torch.tensor(sequences, dtype=torch.long)
        self.y = torch.tensor(labels, dtype=torch.long)
    def __len__(self): return len(self.X)
    def __getitem__(self, idx): return self.X[idx], self.y[idx]

train_ds = DNADataset(X_train, y_train)
test_ds = DNADataset(X_test, y_test)
train_dl = DataLoader(train_ds, batch_size=32, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=32)

# Transformer-based model
class SimpleDNATransformer(nn.Module):
    def __init__(self, vocab_size=5, d_model=64, nhead=4, num_layers=2, max_len=300):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead)
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.classifier = nn.Linear(d_model, 2)

    def forward(self, x):
        x = self.embedding(x).permute(1, 0, 2)  # (seq_len, batch, d_model)
        x = self.encoder(x).permute(1, 2, 0)    # (batch, d_model, seq_len)
        x = self.pool(x).squeeze(-1)            # (batch, d_model)
        return self.classifier(x)               # (batch, 2)

# Initialize
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimpleDNATransformer().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Training loop (few epochs)
for epoch in range(3):
    model.train()
    total_loss = 0
    for xb, yb in train_dl:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        preds = model(xb)
        loss = loss_fn(preds, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1} | Loss: {total_loss:.4f}")

# Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for xb, yb in test_dl:
        xb, yb = xb.to(device), yb.to(device)
        preds = model(xb)
        predicted = preds.argmax(1)
        correct += (predicted == yb).sum().item()
        total += yb.size(0)

print(f"Test Accuracy: {100 * correct / total:.2f}%")


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Epoch 1 | Loss: 158.3539
Epoch 2 | Loss: 155.6938
Epoch 3 | Loss: 153.5260
Test Accuracy: 94.80%


In [8]:
!pip uninstall torch
!pip install torch==2.0.1+cpu torchvision==0.15.2+cpu -f https://download.pytorch.org/whl/torch_stable.html


Found existing installation: torch 2.5.1+cu124
Uninstalling torch-2.5.1+cu124:
  Would remove:
    /usr/local/bin/convert-caffe2-to-onnx
    /usr/local/bin/convert-onnx-to-caffe2
    /usr/local/bin/torchfrtrace
    /usr/local/bin/torchrun
    /usr/local/lib/python3.11/dist-packages/functorch/*
    /usr/local/lib/python3.11/dist-packages/torch-2.5.1+cu124.dist-info/*
    /usr/local/lib/python3.11/dist-packages/torch/*
    /usr/local/lib/python3.11/dist-packages/torchgen/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 MB 8.7 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 57.4 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.5.1+cu124
    Uninstalling torch-2.5.1+cu124:
      Successfully uninstalled torch-2.5.1+cu124
  Attempting uninstall: torchvision
    Found existing installa